In [18]:
n = 3174
dat = genfromtxt('../data/adj.csv',delimiter = ',')
observ = np.array([1,1,0,0,1,1,0])

In [11]:
import numpy as np
from numpy import genfromtxt
transition_probability = dat
emission_probability = np.zeros([n,2])
p = 0.9
for i in range(n):
    emission_probability[i,0] = p
    emission_probability[i,1] = 1-p

In [13]:
def HMMfilter(y, nu, Q, g):
  n = y.size
  k = Q.shape[0]
  phi = np.zeros([k, n])
  c = np.zeros(n)
  
  Z = nu * g[:, y[0]]
  c[0] = sum(Z)
  phi[:,0] = Z/c[0]
  
  for t in range(1,n):
    Z = np.dot(phi[:, t-1], Q) * g[:, y[t]];
    c[t] = sum(Z)
    phi[:, t] = Z / c[t]
  
  return (phi,c)


def HMMsmoother(y, Q, g, c):
  n = y.size
  beta = np.ones([Q.shape[0], n])
  for t in range(n-2, -1, -1):
    beta[:,t] = np.dot(Q, g[:, y[t+1]] * beta[:, t+1]) / c[t+1]
  return beta

def HMMbaumwelch(y, nu, tol=1e-4, maxIt = 100):
  #global myfilter, mysmoother # should be either HMMfilter/HMMsmoother, or HMM_C.HMMfilter/HMM_C.HMMsmoother
  k = nu.size; r = 1+max(y); n = y.size
  Y = np.zeros([n, r]); 
  Y[range(n), np.int_(y)] = 1;
# global var
  Q = transition_probability
  g = emission_probability
# ##
  for j in range(k):
    Q[j,:] = Q[j,:] / sum(Q[j,:])
    g[j,:] = g[j,:] / sum(g[j,:])
  it = 0; oldQ = Q; oldg = g + tol +1
  
  while (sum(sum(abs(oldQ[:]-Q[:]))) + sum(sum(abs(oldg-g))) > tol) & (it<maxIt):
    it+=1
    (phi, c) = HMMfilter(y, nu, Q, g)
    beta = HMMsmoother(y, Q, g, c)
    post = phi * beta
    N = Q * (np.dot(phi[:, 0:-1], np.transpose(beta[:, 1:]*g[:, np.int_(y[1:])]/np.tile(c[1:], [k, 1]))))
    M = np.dot(post, Y)

    oldQ = Q.copy(); oldg = g.copy()
    for j in range(k):
      Q[j,:] = N[j,:] / sum(N[j,:])
      g[j,:] = M[j,:] / sum(M[j,:])
  l = sum(np.log(c))
  return (Q, g, l)

In [ ]:
y = np.asarray(observ)
s =  2803
start_probability = np.zeros([n])
start_probability[s] = 1
#start_probability = np.matrix(start_probability).reshape([-1,1])
#start_probability = start_probability.reshape([-1,1])

nu = start_probability
Q,g,l = HMMbaumwelch(y, nu, tol=1e-4, maxIt = 100)